## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [10]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [11]:
load_dotenv(override=True)

True

In [1]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ygy3389@gmail.com")  # Change to your verified sender
    to_email = To("ygy3389@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

NameError: name 'sendgrid' is not defined

### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [13]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [14]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [15]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC2 Compliance Process with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC2 compliance can be a daunting task. As your organization grows, so do the challenges of maintaining compliance and preparing for audits. At ComplAI, we understand that the traditional methods can be time-consuming and cumbersome.

Our AI-powered SaaS tool is designed to simplify the SOC2 compliance journey. With ComplAI, you can:

- **Automate Documentation**: Reduce manual efforts and streamline document management.
- **Continuous Monitoring**: Stay ahead of compliance requirements with real-time updates and alerts.
- **Audit Preparation**: Simplify the audit process, ensuring you are always ready for review.

We’ve helped companies like [Notable Client] reduce their compliance-related workload by up to 50%, allowing them to focus more on their core business activities.

I’d love to offer you a personalized demonstration of ho

In [16]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC 2 Compliance with AI-Driven Solutions

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I'm reaching out to introduce you to ComplAI, a cutting-edge SaaS tool designed to streamline SOC 2 compliance and prepare your organization for audits.

In today’s rapidly evolving regulatory landscape, achieving and maintaining SOC 2 compliance can be a daunting challenge. ComplAI utilizes advanced AI technology to simplify this process, ensuring your documentation, risk management, and audit preparations are both efficient and effective.

Key Benefits of ComplAI:

1. **Automated Compliance Tracking:** Stay on top of requirements without the manual burden.
2. **Real-Time Insights:** Gain actionable insights to make informed decisions quickly.
3. **Streamlined Audits:** Prepare for audits with certainty, reducing stress and improving outcomes.

I would love the opportunity to discuss how ComplAI can assist [Company Name] in enhancing

In [17]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [18]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: 🚀 Is Your SOC2 Compliance Missing a Rocket Boost?

Hi [Recipient's Name],

I hope this email finds you navigating the vast universe of compliance without getting lost in the black hole of paperwork! 🌌

At ComplAI, we're on a mission to bring your SOC2 compliance process into the future—no more time travel paradoxes (or endless spreadsheets) necessary! With our AI-powered SaaS tool, you can streamline your compliance efforts like a spaceship on hyperdrive. 🛸

Imagine this: audits that don't feel like a root canal, reports that get prepared faster than you can say "SOC2," and peace of mind that comes from knowing you’ll sail right through with confidence.

Interested in how we can help you avoid the compliance asteroid belt? I'd love to schedule a quick chat. Let’s make those audits as easy as pie—without the calories! 🥧

Looking forward to hearing from you!

Best,  
[Your Name]  
[Your Position]  
ComplAI  
[Your Contact Information]  

P.S. If you respond, I 

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [19]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [20]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [33]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ygy3389@gmail.com")  # Change to your verified sender
    to_email = To("ygy3389@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [34]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000262889D9BC0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [35]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026288AB8680>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [36]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026288AB8FE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026287E27E20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

## And now it's time for our Sales Manager - our planning agent

In [37]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [45]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [46]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ygy3389@gmail.com")  # Change to your verified sender
    to_email = To("ygy3389@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [47]:
tools = [subject_tool, html_tool, send_html_email]

In [48]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026288ABA020>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026288AB9A80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to 

In [49]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [50]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026288AB8FE0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026287E27E20>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [51]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

### 1. 识别使用的Agentic设计模式（Can you identify the Agentic design patterns that were used here?）

笔记本展示了几个经典的代理设计模式，这些模式让系统从简单的工作流演变为灵活的代理协作。以下是主要模式，结合代码示例：

- __工具使用模式 (Tool Use Pattern)__：

  - __描述__：代理使用工具（functions或agents-as-tools）来执行外部操作或子任务，而不是仅靠LLM生成文本。这允许代理“行动”，如调用API发送邮件。
  - __笔记本示例__：销售经理代理使用三个sales_agent工具（`tool1, tool2, tool3`）生成邮件草稿，并使用`@function_tool`装饰的`send_email`或`send_html_email`发送邮件。代码中：`tools = [tool1, tool2, tool3, send_email]`。这让代理能“思考”（规划）+“行动”（工具调用）。
  - __为什么是模式__：这是基础代理模式（如OpenAI/LangChain中常见），代理循环：LLM决定工具调用 → 执行工具 → 返回结果到LLM → 迭代直到完成。

- __代理作为工具模式 (Agents-as-Tools Pattern)__：

  - __描述__：将一个代理包装成工具，让另一个代理调用它作为子模块，实现模块化协作。
  - __笔记本示例__：三个销售代理被转换为工具：`tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")`。销售经理通过这些工具并行生成草稿（`await asyncio.gather(...)`），然后内部评估挑选最佳的。这比手动并行运行更“代理化”。
  - __为什么是模式__：它允许层次化（hierarchical）代理，父代理统筹子代理，类似于“微服务”架构。

- __Handoff/委托模式 (Handoff/Delegation Pattern)__：

  - __描述__：代理将控制权完全移交给另一个专用代理处理子任务，支持自治协作，而非微观管理。
  - __笔记本示例__：最终版本中，销售经理生成草稿后，将最佳邮件`handoff`给`emailer_agent`（`handoffs = [emailer_agent]`，描述为"Convert an email to HTML and send it"）。emailer_agent自己使用`subject_writer`、`html_converter`工具写主题、转换HTML并发送。这引入了“跨代理”控制流。
  - __为什么是模式__：类似于ReAct（Reasoning + Acting）或CrewAI的角色分工，代理团队像“接力赛”。

- __规划-执行-评估循环模式 (Plan-Execute-Review Loop)__：

  - __描述__：代理先规划步骤、执行工具，然后评估输出，可能迭代改进。
  - __笔记本示例__：销售经理指令明确步骤：1. 生成草稿（用工具）；2. 评估选择最佳；3. handoff发送。早期版本有`sales_picker`代理评估，但后期整合到经理中，支持迭代（指令允许“use tools multiple times if not satisfied”）。
  - __为什么是模式__：这是反射（Reflection）或自省（Self-Reflection）模式的变体，代理能“反思”输出质量。

- __并行执行模式 (Parallel Execution Pattern)__：

  - __描述__：代理同时调用多个工具/子代理，提高效率。
  - __笔记本示例__：`await asyncio.gather(Runner.run(sales_agent1, message), ...)` 并行生成三个草稿。
  - __为什么是模式__：常见于多代理系统中，减少延迟。

总体，这些模式让系统像一个“销售开发代表（SDR）”团队：经理规划，销售员生成，格式化员发送。笔记本从简单并行（无模式）逐步添加这些，展示了代理设计的演进。


### 2.哪一行代码将这个从Agentic "workflow"转变为"agent"（What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?）

- __Anthropic定义简述__：根据Anthropic（Claude的开发者）的代理观，"workflow" 是固定、 predictable 的步骤链（如脚本化工具调用，无LLM决策灵活性）。而"agent" 需&#x8981;__&#x81EA;治（autonomy）__：LLM能动态决策、迭代、处理意外（如反思输出、使用工具自主），而非刚性流程。这通常通过工具+循环+委托实现，允许代理“像人类一样”适应（参考Anthropic的Tool Use和Agentic Patterns文档，强调“reasoning loops”和“delegation”）。

- __关键一行代码__：添加手off到代理初始化中，即：

  ```javascript
  handoffs=[emailer_agent]
  ```

  - __位置__：在最终`sales_manager`代理的定义中：`sales_manager = Agent(..., handoffs=handoffs, model="gpt-4o-mini")`，其中`handoffs = [emailer_agent]`。

  - __为什么是这行__？

    - __之前是workflow__：在"Part 2: use of tools"，销售经理自己处理一切（生成→挑选→发送），这是固定循环：工具调用 + 内部评估。像脚本：步骤预定义，无真正委托。
    - __转变为agent__：添加handoff后，经理能动态委托子任务给emailer_agent（另一个自治代理），并让它独立运行自己的工具链（写主题→转换HTML→发送）。这引&#x5165;__&#x81EA;治协作__：经理不控制细节，而是“交棒”，允许系统处理变异（如emailer_agent可迭代改进HTML）。这符合Anthropic的"agent"定义——有动态决策和子代理自治，而非纯workflow。
    - __影响__：系统从单代理脚本变成多代理团队，LLM能“反思”何时handoff（基于指令），支持Anthropic强调的“extended reasoning”和“delegation for complex tasks”。

如果没有handoff，系统仍是“工具工作流”；handoff让它“活起来”，成为真正代理。


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.